## OpenAI (GPT-4o-mini)

GPT-4o-miniに直接画像とプロンプトを投げる

API Reference
- https://platform.openai.com/docs/api-reference/chat
- https://platform.openai.com/docs/guides/vision

In [24]:
import os
from dotenv import load_dotenv
import base64
import re
import json

import pandas as pd

import openai
from openai import OpenAI

In [2]:
!ls ../menu_images

menu1.jpg menu2.jpg menu3.jpg menu4.jpg


In [3]:
image_dir = '../menu_images'
images = ['menu1.jpg', 'menu2.jpg', 'menu3.jpg', 'menu4.jpg']

In [4]:
load_dotenv('.env')

client = OpenAI(api_key=os.getenv('openai-key-20241027'))
client

In [5]:
%%time

responses = {}

vision_model = "gpt-4o-mini"

prompt = '''
メニュー画像が入力されます。記載されているメニュー情報全てを抽出するのが目的です。「メニュー名」「内容説明」「価格」を以下のjson形式で出力してください。記載がない部分は「null」と出力してください。

[{
"メニュー名": "'唐揚げ'等のメニューの名前",
"内容説明": "メニュー内容の説明。記載がなければ'null'とする",
"価格": "メニューに対応した価格",
}]
'''

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

for img in images:
    print('>>>>> ', img)
    
    # Path to your image
    image_path = os.path.join(image_dir, img)
    
    # Getting the base64 string
    base64_image = encode_image(image_path)
    #print(type(base64_image), len(base64_image))

    response = client.chat.completions.create(
        model=vision_model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            }
        ],
    )
    responses[img] = response.choices[0].message.content
    #print(responses[img])

>>>>>  menu1.jpg
>>>>>  menu2.jpg
>>>>>  menu3.jpg
>>>>>  menu4.jpg
CPU times: user 90.6 ms, sys: 28.3 ms, total: 119 ms
Wall time: 1min 14s


In [6]:
#print(prompt)

In [8]:
for k,v in responses.items():
    print('>>>>> {}'.format(k))
    print(v)

>>>>> menu1.jpg
```json
[
    {
        "メニュー名": "ダイナソー1ポンドステーキ",
        "内容説明": "大迫力！1ポンド(450g)のステーキ。焼き加減はレア・ミディアム・ウェルダンより1つお選びください",
        "価格": "3769円"
    },
    {
        "メニュー名": "ダイナソー1ポンドカットステーキ",
        "内容説明": "1/2ポンド(225g)のホ切ステーキ。焼き加減はレア・ミディアム・ウェルダンより1つお選びください",
        "価格": "2213円"
    },
    {
        "メニュー名": "ダイナソー1ポンドハンバーグ",
        "内容説明": "オリジナルハンバーグを豪華に3枚！1ポンド(450g)の迫力ハンバーグ",
        "価格": "2504円"
    },
    {
        "メニュー名": "ダイナソーBBQグリル",
        "内容説明": "スペアリブをトロトロになるまで煮込んで焼き上げました！700g(食込)",
        "価格": "3769円"
    },
    {
        "メニュー名": "本日の肉付き肉（要予約）",
        "内容説明": "現代肉と肉の良さの合併肉！贅沢にふっ切り！1kg",
        "価格": "7564円"
    },
    {
        "メニュー名": "テンダーロインステーキ",
        "内容説明": "やわらかさが自慢のステーキ 200g",
        "価格": "3769円"
    }
]
```
>>>>> menu2.jpg
```json
[
    {
        "メニュー名": "本日のカルパッチョ",
        "内容説明": "null",
        "価格": "638"
    },
    {
        "メニュー名": "サーモンとアボカドのクリーミーきのこソース",
        "内容説明": "null",
        "価格": "693"
    },
    {
      

In [15]:
#responses['menu4.jpg']

In [26]:
responses2 = {}
dfs = {}
for k,v in responses.items():
    print('>>>>> {}'.format(k))

    # 不要文字を削除。Jason以外の一番外[]の「```json」とか
    cleaned_str = re.sub(r'^```json\n|\n```$', '', v)
    responses2[k] = cleaned_str

    # Pandasへ変換
    menu_data = json.loads(responses2[k])
    dfs[k] = pd.DataFrame(menu_data)
    

>>>>> menu1.jpg
>>>>> menu2.jpg
>>>>> menu3.jpg
>>>>> menu4.jpg


In [28]:
#responses2['menu1.jpg']

In [23]:
#print(responses2['menu1.jpg'])

In [29]:
for k,v in dfs.items():
    print('>>>>> ', k)
    display(v)

>>>>>  menu1.jpg


,メニュー名,内容説明,価格
0,ダイナソー1ポンドステーキ,大迫力！1ポンド(450g)のステーキ。焼き加減はレア・ミディアム・ウェルダンより1つお選び...,3769円
1,ダイナソー1ポンドカットステーキ,1/2ポンド(225g)のホ切ステーキ。焼き加減はレア・ミディアム・ウェルダンより1つお選び...,2213円
2,ダイナソー1ポンドハンバーグ,オリジナルハンバーグを豪華に3枚！1ポンド(450g)の迫力ハンバーグ,2504円
3,ダイナソーBBQグリル,スペアリブをトロトロになるまで煮込んで焼き上げました！700g(食込),3769円
4,本日の肉付き肉（要予約）,現代肉と肉の良さの合併肉！贅沢にふっ切り！1kg,7564円
5,テンダーロインステーキ,やわらかさが自慢のステーキ 200g,3769円


>>>>>  menu2.jpg


,メニュー名,内容説明,価格
0,本日のカルパッチョ,null,638
1,サーモンとアボカドのクリーミーきのこソース,null,693
2,マグロとイカのカルパッチョ,null,null
3,パリパリチキンの彩りズザ,null,null
4,ザンギ,null,528
5,ラムの鉄板焼き,null,null
6,焼豚マスタードソース,null,null
7,ポテトフライ,null,440
8,アンチョビポテト,null,506
9,生ハムのブレスケッタ,null,627


>>>>>  menu3.jpg


,メニュー名,内容説明,価格
0,オムライス,omlet with a filling of ketchup-seasoned fried...,900
1,大盛オムライス,large serving of omelet with a filling of ketc...,1300
2,エビフライ,fried prawns,1100
3,ポークカツ,pork cutlet,1100
4,ビーフカレー,beef curry,900
5,ビーフシチュー,beef stew,1300
6,ハンバーグ,hamburger steak,1100
7,コーンスープセット,corn soup set,800
8,ポークジャンボ,Pork Jamboree (Pork Singer Diner),900
9,和風海鮮丼,Japanese-style seafood Donburi,null


>>>>>  menu4.jpg


,メニュー名,内容説明,価格
0,焼き芋と蒸しレンコンのソイマスタードサラダ,長時間蒸し上げた芋とレンコンとサクサクのクルミをしっかりと混ぜリズミカルな表情です。,¥1851
1,ローストチキンのチキンチャップサラダ,専門店の味を自宅で楽しむセットメニューです。,¥1901
2,きのことかぼちゃのクリームパスタ,生クリームの本分で甘みが引き立った季節のきのこを使用したクリームパスタ。,¥1610
3,有機ミニサラダセット,サラダと主菜、パンのセットとなります。,¥1851
4,自家製ししおのチューリップ唐揚げ,天然の味付けが豊かな、食べごたえのある唐揚げです。,null
5,平飼い鶏季節野菜のサラダ煮込み,平飼いの鶏と旬の野菜が優しく煮込まれています。,null
6,キッズプレート,お子様のお誕生日や特別な日に。,¥651
7,豆とトマトのVEGANカレー,厳選した野菜を使ったカレー。,null
8,大豆ミート唐揚げ油淋鶏風,サクサクとした食感の代替肉を使った唐揚げ風。,null
9,スープ,旬野菜をたっぷり取り入れた温かいスープ。,¥401


In [32]:
save_dirs = ['./results/menu1/', './results/menu2', './results/menu3', './results/menu4']
save_dirs_files = [os.path.join(d, 'replyOAI_VM.csv') for d in save_dirs]

i = 0
for k,v in dfs.items():
    print('>>>>> ', k)
    v.to_csv(save_dirs_files[i], index=False)
    i += 1

>>>>>  menu1.jpg
>>>>>  menu2.jpg
>>>>>  menu3.jpg
>>>>>  menu4.jpg
